In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from pytorch_lightning import Trainer
from torch.nn import functional as F
from torchvision.utils import make_grid

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
from dhdrnet.util import ROOT_DIR

data_dir = ROOT_DIR / "precomputed_data"

In [9]:
df = pd.read_csv(data_dir / "store_finer_2020-07-06.csv", index_col="Unnamed: 0")
df

/home/shane/.cache/pypoetry/virtualenvs/dhdrnet-CxDQYEDG-py3.8/lib64/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,name,metric,ev1,ev2,score
0,0009_20160703_160626_325,mse,-3.00,-2.75,9546.539856
1,0009_20160703_160626_325,ssim,-3.00,-2.75,0.501896
2,0009_20160703_160626_325,perceptual,-3.00,-2.75,0.224065
3,0009_20160703_160626_325,mse,-3.00,-2.50,8784.144451
4,0009_20160703_160626_325,ssim,-3.00,-2.50,0.540524
...,...,...,...,...,...
688,33TJ_20150730_201700_327,ssim,5.50,6.00,0.514825
689,33TJ_20150730_201700_327,perceptual,5.50,6.00,0.536411
690,33TJ_20150730_201700_327,mse,5.75,6.00,10301.445653
691,33TJ_20150730_201700_327,ssim,5.75,6.00,0.489459


In [16]:
no_synth_df = df[~df["name"].str.startswith("synthetic")].copy()

In [20]:
no_synth_df.to_csv(data_dir / "store_no_synth.csv", index=False)

In [21]:
nsdf = pd.read_csv(data_dir / "store_no_synth.csv")
nsdf

,name,metric,ev1,ev2,score
0,0009_20160703_160626_325,mse,-3.00,-2.75,9546.539856
1,0009_20160703_160626_325,ssim,-3.00,-2.75,0.501896
2,0009_20160703_160626_325,perceptual,-3.00,-2.75,0.224065
3,0009_20160703_160626_325,mse,-3.00,-2.50,8784.144451
4,0009_20160703_160626_325,ssim,-3.00,-2.50,0.540524
...,...,...,...,...,...
7230757,33TJ_20150730_201700_327,ssim,5.50,6.00,0.514825
7230758,33TJ_20150730_201700_327,perceptual,5.50,6.00,0.536411
7230759,33TJ_20150730_201700_327,mse,5.75,6.00,10301.445653
7230760,33TJ_20150730_201700_327,ssim,5.75,6.00,0.489459


In [25]:
names = pd.Series(pd.unique(nsdf["name"]))

0       0009_20160703_160626_325
1       0043_20160825_125822_548
2       0006_20160728_123257_859
3       33TJ_20150705_191438_366
4       0127_20161022_152006_986
                  ...           
3614    6G7M_20150325_105827_075
3615    0043_20160923_122514_781
3616    0155_20160810_215900_160
3617    0382_20150924_090901_068
3618    33TJ_20150730_201700_327
Length: 3619, dtype: object

In [28]:
names_train = names.sample(frac=0.8)
names_test = names.loc[names.index.difference(names_train.index)]
names_train.to_csv(ROOT_DIR / "train_no_synthetic.csv", index=False, header=["names"])
names_test.to_csv(ROOT_DIR / "test_no_synthetic.csv", index=False, header=["names"])